In [1]:
import torch
print(torch.__version__)         # 2.7.1+cu118
print(torch.version.cuda)        # 11.8
print(torch.cuda.is_available()) # True → GPU 사용 가능
print(torch.cuda.get_device_name(0))


2.4.1+cu121
12.1
True
NVIDIA GeForce RTX 4090


### Tokenizer Test

In [2]:
# !pip install transformers
from transformers import AutoTokenizer

prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids
for t in input_ids:
    print(t,"\t:", tokenizer.decode(t))


/home/gpuadmin/anaconda3/envs/sdxl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


### AutoModelForCausalLM test
- 위 예시와 아래의 예시에서 AutoTokenizer 와 AutoModelForCausalLM 을 사용했다는 점에 주목합시다.
- transformers 라이브러리에서는 수백 개의 모델과 대응하는 토크나이저를 지원합니다. AutoTokenizer 와 AutoModelFor* 을 활용해 다양한 모델을 쉽게 로드해 사용할 수 있습니다.
- 단, 무슨 모델은 불러올지는 명시해야합니다. 아래처럼 Qwen2 0.5b 모델을 사용할때는 AutoModelForCausalLM, 분류 모델을 불러올때는 AutoModelForSequenceClassification, 객체 탐지에는 AutoModelForObjectDetection을 사용할 수 있습니다. 각 라이브러리 명을 보면 대략적으로 무슨 모델을 로드할때 사용할 지 판단할 수 있습니다.

In [3]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

# 초크나이저를 다시 호출하되, pytorch 텐서를 반환하도록 지정
# 모델이 정수 리스트 대신 파이토치 텐서를 입력으로 받음

input_ids = tokenizer(prompt, return_tensors='pt').input_ids
output = model(input_ids)
output.logits.shape


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


torch.Size([1, 7, 151936])

- 출력의 첫 번째 차원은 배치 크기를 나타내고 우리는 한 문장만 전달 했기에 그 크기는 1입니다.
- 두 번째 차원은 문장의 길이, 입력 문장의 토큰 수를 나타냅니다. 토크나이저마다 문장을 slice하는 기준이 다르기에 그 값은 같은 문장이라도 조금씩 다를 수 있습니다.
- 세 번째 차원은 어휘 사전 크기를 나타냅니다.
- 이러한 값들은 어휘 사전의 토큰에 대응하는 모델의 초기 출력 값인 logit으로 [0.1, -2.1, 1.2, 0.01 ...] 같은 숫자 리스트입니다. 이 logit을 사용해 다음 이어질 확률이 가장 높은 토큰을 선택할 수 있고 logit을 확률로 변환하는 방법도 있습니다.

In [4]:
final_logits = model(input_ids).logits[0,-1]
final_logits.argmax()

tensor(3729)

- 3729는 모델이 'It was a dark and stormy'라는 문장을 보고 다음에 올 가능성이 가장 높은 단어를 나타내는 일종의 ID값입니다. 이를 decode하면 ' night'라는 단어가 나오며 이를 통해 모델이 다음 흐름을 어떤 단어로 학습하고 산출했는지 알 수 있습니다.

In [5]:
len(final_logits)

151936

- 전체 151936개 중 가장 큰 값임을 알 수 있습니다.

In [6]:
tokenizer.decode(final_logits.argmax())

' night'

- 상위 10개의 확률 분포를 볼 수 있습니다. 전체 어휘 사전의 확률을 모두 더하면 100이 됩니다. 모든 어휘가 확률을 가지고 있으나 상위를 제외하고는 매우 낮습니다.

In [7]:
import torch
# top10_logits = torch.topk(final_logits,10)
# for i in top10_logits.indices:
#     print(tokenizer.decode(i))

top10 = torch.topk(final_logits.softmax(dim=0),10)
for v,i in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(i):<10} {v.item():.3%}")

 night     88.706%
 evening   4.300%
 day       2.192%
 morning   0.487%
 winter    0.449%
 afternoon 0.272%
 Saturday  0.252%
 Sunday    0.187%
 Friday    0.171%
 October   0.165%


### 단어 변경하기, 입력 문자열 변경하기, 문법 오류 실험해보기

In [8]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


def test_process(prompt:str):
    prompt = prompt
    # tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
    # input_ids = tokenizer(prompt).input_ids
    # input_ids
    # for t in input_ids:
    #     print(t,"\t:", tokenizer.decode(t))
        

    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
        
    final_logits = model(input_ids).logits[0,-1]
    final_logits.argmax()

    top10 = torch.topk(final_logits.softmax(dim=0),10)
    for v,i in zip(top10.values, top10.indices):
        print(f"{tokenizer.decode(i):<10} {v.item():.3%}")

case = ["It was a stormy and dark", "What's wrong with you? Why", "It were and and and"]

for c in case:
    print(c)
    test_process(c)
    print()

It was a stormy and dark
 night     67.607%
 day       6.383%
 evening   6.228%
 morning   2.221%
 summer    1.482%
 afternoon 1.437%
 winter    1.432%
 Saturday  0.931%
 weekend   0.827%
 Sunday    0.788%

What's wrong with you? Why
 are       27.435%
 do        15.870%
 don       12.520%
 can       6.286%
 did       5.347%
 didn      3.023%
 aren      2.746%
 you       2.705%
 not       2.338%
 have      1.965%

It were and and and
 and       27.701%
 I         4.080%
 the       3.245%

          2.408%
 a         1.814%
 ,         1.733%
 it        1.629%
,          1.543%
 .         1.424%
 that      1.393%



- 모델이 문장에서 다음 토큰을 예측 하는 방법을 알게 되면, 예측 결과를 모델에 반복해서 전달해 문장을 생성할 수 있다.
- model(ids)를 호출해 새 토큰 ID를 생성하고 이를 리스트에 추가한 뒤, 다시 함수를 호출하면 된다.
- 트랜스포머의 자기회귀 모델은 generate() 메소드를 사용한다.
- 이를 탐욕적 디코딩(greedy decoding) 이라고 하며, 이런 경우 바로 다음 단어에만 집중해 답을 선택하기 때문에 전체적으로 자연스러운 문장을 놓지고 부자연스러운 단어 조합만 얻게 될 수도 있다.

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

# 입력 문장
text = "It was a dark and stormy night"
inputs = tokenizer(text, return_tensors="pt")

# attention_mask와 pad_token_id 추가
output_ids = model.generate(
    inputs["input_ids"],
    max_new_tokens=28,
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id  # GPT류 모델일 경우 안전
)

# 디코딩
decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("input ids:", inputs["input_ids"][0])
print("output ids:", output_ids[0])
print("gen text :", decoded_text)

input ids: tensor([ 2132,   572,   264,  6319,   323, 13458,    88,  3729])
output ids: tensor([ 2132,   572,   264,  6319,   323, 13458,    88,  3729,    13,   576,
        12884,   572,  6319,   323,   279,  9956,   572,  1246,  2718,    13,
          576, 11174,   572, 50413,  1495,   323,   279, 32438,   572, 49757,
           13,   576, 12884,   572,  6319,   323])
gen text : It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the lightning was flashing. The sky was dark and


- 탐욕적 디코딩 대신, beam search 같은 기법으로 여러 후보 문장들을 탐색한 후 가장 적절한 결과를 반환한다. 

In [13]:
beam_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    num_beams=5,
    max_new_tokens= 30
)

print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

It was a dark and stormy night. The wind was howling, and the rain was pouring down. The sky was dark and gloomy, and the ground was wet and muddy.


- 모델에 따라 다소 반복되는 결과가 나올 수 도 있다. 이를 피하기 위한 여러 매개변수 중 두 가지가 있다.
- repetition_penalty : 이미 생성된 토큰에 패널티를 부여해 반복을 방지한다. 적정 계수는 1.2 정도라고 한다.
- bad_words_ids : 생성되지 않아야 하는 단어 목록을 지정한다.

In [ ]:
beam_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    num_beams=5,
    max_new_tokens= 30,
    repetition_penalty = 2.0,
    #bad_words_ids
)

print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

It was a dark and stormy night. The sky was filled with thunder and lightning, and the wind howled in the distance. It was raining cats and dogs, and the streets were


- do_sample=True, # 샘플링을 하여 좀 더 무작위성을 넣는다.
- temperature = 0.4, # 분포를 더 좁거나 평평하게 하는 변수, 1보다 크면 무작위성이 높아지고 1보다 작아지면 무작위성이 낮아진다. 0이라면 반드시 가장 확률이 높은 토큰으로 집중된다.
- top_k = 20, # 가능성 상위 n개의 토큰에서만 샘플링을 진행한다. 위 무작위성의 매개 변수에 제약을 둔다.
- top_p = 80, # 가능성 상위 n%를 넘는 토큰에서만 샘플링을 진행한다. 위 무작위성의 매개 변수에 제약을 둔다.

In [ ]:
from transformers import set_seed

set_seed(70)

sampling_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature = 0.4, 
    top_k = 0,
    # num_beams=5,
    max_new_tokens= 40,
    # repetition_penalty = 2.0,
    #bad_words_ids
)
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night, and the last thing I wanted to do was walk home. I was tired and hungry. I was tired of being alone and didn't want to be alone in the dark. I had been walking


In [ ]:
from transformers import set_seed

set_seed(70)

sampling_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature = 3.0, 
    top_k = 0,
    # num_beams=5,
    max_new_tokens= 40,
    # repetition_penalty = 2.0,
    #bad_words_ids
)
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night 不公布 Injectorstaff Manipcad       
<Appmoduleisodes -->
сим свою çı.val */
 argc	Route я Hopieverului hur'].' sulla grains enrich PROGRAM vminiese Increases Butterfly Up Farms из MechanenumFun drills+'_


In [21]:
from transformers import set_seed

set_seed(70)

sampling_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature = 1.0, 
    top_k = 20,
    # num_beams=5,
    max_new_tokens= 40,
    # repetition_penalty = 2.0,
    #bad_words_ids
)
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night, and the entire world was a quiet graveyard. The moon was dark, and the stars were gone from the sky. It was the night of the most terrible night of my life.
"Hey,


In [22]:
from transformers import set_seed

set_seed(70)

sampling_output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature = 1.0, 
    top_p = 80,
    # num_beams=5,
    max_new_tokens= 40,
    # repetition_penalty = 2.0,
    #bad_words_ids
)
print(tokenizer.decode(sampling_output[0]))

It was a dark and stormy night, and the entire world was a quiet graveyard. The house the two children were buried in was located in the heart of the city, right next to the house in which Father Harte had lived.


### 제로샷 일반화

- 모델에 특정 요구사항을 전달하면 그대로 작업을 진행해준다. 아래의 예시를 보면

In [43]:
def score(review):
    prompt = f"""Question: IS the following review Positive or Negative about the movie? 
    Review: {review} Answer:"""
    input_ids = tokenizer(prompt,return_tensors="pt").input_ids
    final_logits = model(input_ids).logits[0,-1]
    if final_logits[35490] <= final_logits[42224]: # 35490는 psitive라는 단어의 토큰 id이고 42224 는 negative 라는 단어의 토큰 id이다.
        print('Negative')
    else:
        print('Positive')

score("That movie was terrible! I hate it!")
score("I love this movie. But some scenes are not good. Anyway the main character was so cute! Good movie!")

Negative
Positive


- 이외에도 몇줄의 영어를 스페인어로 번역하는 예시를 주고 작업을 지시하는 "fewshot 일반화" 라는 기법으로 유사한 결과를 얻을 수 있다.